In [1]:
import os
import cv2
import json
import glob
import shutil
import time

import PBL_detection
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PBL_detection import *
from Roots_point_detect_tool import*
from pattern_detection_tool import*
from tqdm.notebook import tqdm as tqdm

In [2]:
start = 0
end = 500

In [3]:
step = 2

In [4]:
assert step==1,'modify step'

AssertionError: modify step

In [ ]:
# def folder_builder(parent_path,folder_name,Rebuild):
#     if os.path.exists(os.path.join(parent_path, folder_name)) and not Rebuild:
#         print('"{}" folder is exist,if want to rebuild please modify "Rebuild" parameter.'.format(folder_name))
#     else:
#         if os.path.exists(os.path.join(parent_path, folder_name)):
#             shutil.rmtree(os.path.join(parent_path, folder_name))
#         try:
#             os.makedirs(os.path.join(parent_path, folder_name))
#             print('"{}" folder build successful'.format(folder_name))
#         except:
#             print('folder build error')

In [ ]:
#First time Rebuild = True
Rebuild = True
folder_build_path = './'
# sub_folder_name_list = ['1_o_image','1_roots_mask','1_seg_image','2_seg_img_with_cej',
#                         '3_pattern_detection_result','3_draw_all_points_on_o_img',
#                         '3_draw_all_points_on_seg_img','4_result_with_label',
#                         'csv_file','Result_comparison']
sub_folder_name_list = ['1_o_image','1_roots_mask','1_seg_image','2_seg_img_with_cej',
                        '3_pattern_detection_result','3_draw_all_points_on_o_img',
                        '3_draw_all_points_on_seg_img','4_result_with_label',
                        ]
if not Rebuild and os.path.exists(folder_build_path):
    print('This folder is exist,if want to rebuild please modify "Rebuild" parameter.')
else:
    for sub_folder_name in sub_folder_name_list:
        folder_builder(folder_build_path,sub_folder_name,Rebuild)
    print('Initial the all folder.')

In [ ]:
# seg_img_list = glob.glob('./preprocess/seg_img/*.PNG')
# seg_img_list.sort()

# o_img_path_list = glob.glob('./preprocess/o_img/*.PNG')
# print(len(o_img_path_list))
# root_img_path_list = glob.glob('./preprocess/root_mask_img/*.PNG')
# print(len(root_img_path_list))

# o_img_save_path = './1_o_image'
# root_save_path = './1_roots_mask'
# seg_img_save_path = './1_seg_image'

# count=0

# for seg_img_path in tqdm(seg_img_list,total=len(seg_img_list)):
#     image_name = seg_img_path.split('/')[-1].split('.')[0]
#     o_img_path = [img_path for img_path in o_img_path_list if image_name in img_path]
#     root_img_path = [img_path for img_path in root_img_path_list if image_name in img_path]
    
#     if o_img_path==[] or root_img_path==[]:
#         count+=1
#         pass
#     for img in o_img_path:
#         img_shape = cv2.imread(img).shape 
#         shutil.copy(img,os.path.join(o_img_save_path,image_name+'.PNG'))
        
#     seg_resize = cv2.resize(cv2.imread(seg_img_path),(img_shape[1],img_shape[0]),interpolation=cv2.INTER_NEAREST)
#     cv2.imwrite(os.path.join(seg_img_save_path,image_name+'.PNG'),seg_resize)
# #     for img in root_img_path:
# #         root_name = img.split('/')[-1].split('.')[0]
# #         root = cv2.imread(img)
# #         root_resize = cv2.resize(root,(img_shape[1],img_shape[0]),interpolation=cv2.INTER_NEAREST)
# #         cv2.imwrite(os.path.join(root_save_path,root_name+'.PNG'),root_resize)
# print(count)

In [ ]:
# seg_img_list = glob.glob('./preprocess/seg_img/*.PNG')
# seg_img_list.sort()

# o_img_list = glob.glob('./preprocess/o_img/*.PNG')
# o_img_list.sort()

# root_img_list = glob.glob('./1_roots_mask/*.PNG')
# root_img_list.sort()
# print(len(seg_img_list),len(o_img_list),len(root_img_list))

# o_img_save_path = './1_o_image'
# seg_img_save_path = './1_seg_image'

# count = 0

# for o,s in tqdm(zip(o_img_list,seg_img_list),total=len(seg_img_list)):
#     o_img = cv2.imread(o)
#     s_img = cv2.imread(s)
#     assert o.split('/')[-1] == s.split('/')[-1],'fuck you'
#     img_name = o.split('/')[-1][:-4]
#     try:
#         target_img = [img_path for img_path in root_img_list if img_name in img_path] 
#         target_size = cv2.imread(target_img[0]).shape
#         o_img_resize = cv2.resize(o_img,(target_size[:2][::-1]), interpolation=cv2.INTER_NEAREST)
#         s_img_resize = cv2.resize(s_img,(target_size[:2][::-1]), interpolation=cv2.INTER_NEAREST)
#         cv2.imwrite(os.path.join(o_img_save_path,o.split('/')[-1]),o_img_resize)
#         cv2.imwrite(os.path.join(seg_img_save_path,s.split('/')[-1]),s_img_resize)
#     except:
#         print(img_name)
#         count+=1

In [ ]:
assert step==2,'modify step can run step 2'

In [ ]:
#STEP 2 Using split root_dilation to be mask,shift the root to match cej,
folder_build_path = './'

seg_image_folder = os.path.join(folder_build_path,'1_seg_image')
seg_image_path_list = glob.glob(seg_image_folder+'/*.PNG')
seg_image_path_list.sort()

split_root_folder = os.path.join(folder_build_path,'1_roots_mask')
split_root_path_list = glob.glob(split_root_folder+'/*.PNG')
split_root_path_list.sort()

seg_img_with_cej_save_path = os.path.join(folder_build_path,'2_seg_img_with_cej')

have_save_path = os.path.join(folder_build_path,'2_seg_img_with_cej')
have_save_path_list = glob.glob(have_save_path+'/*.PNG')
have_save_path_list.sort()

# if show,save = true would show,save image what this block create
show = False
save = True
save_file_type = '.PNG'
print_ = False


seg_image_path_list = seg_image_path_list[:end]

for index, seg_image_path in tqdm(enumerate(seg_image_path_list), total = len(seg_image_path_list)):
    img_name = get_name_from_path(seg_image_path,False)
    if print_:
        print(img_name)
    roots_img_path_list = [root_name for root_name in split_root_path_list if img_name in root_name]
    have_fill_num = [root_name for root_name in have_save_path_list if img_name in root_name]
    
#     if len(roots_img_path_list)==len(have_fill_num):
#         continue
    
    o_img = cv2.imread(seg_image_path.replace('1_seg_image','1_o_image'),0)    
    seg_img = cv2.imread(seg_image_path)

    if print_:
        print(show_pixel_set(seg_img))

    #seg_img = cv2.cvtColor(seg_img, cv2.COLOR_BGR2RGB)
    seg_img_copy = seg_img.copy()
    seg_img_copy_gray = cv2.cvtColor(seg_img_copy, cv2.COLOR_BGR2GRAY)
    
    tooth_mask = np.where(seg_img_copy_gray==226,1,0).astype('uint8')
    forward, value = tooth_forward(seg_img)
    
    if print_:
        check_status(value)
        
    if show:
        plt.subplot(121)
        plt.title('seg_img')
        plt.imshow(seg_img_copy)
        plt.subplot(122)
        plt.title('o_img')
        plt.imshow(o_img, cmap = 'gray')
        plt.show()
    for o_root_path in roots_img_path_list:
        root_name = o_root_path.split('/')[-1]
        o_root_img = cv2.imread(o_root_path,0)
        o_root_img = cv2.resize(o_root_img,o_img.shape[:2][::-1],interpolation = cv2.INTER_NEAREST)
        #fill_up_ = img_dilation(o_root_img,tooth_mask,10)
        fill_up_ = get_boundary_line_by_inertia(o_root_img, tooth_mask)
        seg_img_with_cej = combine_seg_cej(seg_img_copy,fill_up_).astype('uint8')
        if save:
            cv2.imwrite(os.path.join(seg_img_with_cej_save_path,root_name), seg_img_with_cej)
        if show:
            plt.figure(figsize=(12,12))
            plt.subplot(141)
            plt.title('fill_up_')
            plt.imshow(fill_up_)
            plt.subplot(142)
            plt.title('tooth_mask')
            plt.imshow(tooth_mask, cmap = 'gray')
            plt.subplot(143)
            plt.title('o_root_img')
            plt.imshow(o_root_img, cmap = 'gray')
            plt.subplot(144)
            plt.title('seg_img_with_cej')
            plt.imshow(cv2.cvtColor(seg_img_with_cej,cv2.COLOR_BGR2RGB))
            plt.show()

In [ ]:
# assert step==3,'modify step can run step 3'

In [5]:
#initial csv file
Resave = True
csv_file_path = './PBL_result_1.csv'

if not Resave and os.path.exists(csv_file_path):
    print('This csv file is exist,if want to resave please modify "Resave" parameter.')
else:
    print('Initial the csv.')
    with open(csv_file_path, 'w', newline='\n') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Image_name', 'Tooth_num', 'LPBL(%)', 'L_Stage(int)', 'RPBL(%)', 'R_Stage(int)','Dis_L','Dis_R','Dis_root','points_info','tooth_pbl_line_path'])

Initial the csv.


In [6]:
# def PBL_Detect(fill_up_img,imgarray,roots_point,cej_points,bone_points,line_of_inertia):
#     print_ = False
#     roots_point = list(sorted(roots_point, key = lambda x:x[0]))
#     cej_points = list(sorted(cej_points, key = lambda x:x[0]))
#     bone_points = list(sorted(bone_points, key = lambda x:x[0]))
#     RGB_image = cv2.cvtColor(imgarray, cv2.COLOR_GRAY2RGB)
#     img_PBL_line = RGB_image.copy()
#     L_side = False
#     R_side = False
#     cej_num_list = [point_in_left_side_of_line(cej,line_of_inertia[0],line_of_inertia[1]) for cej in cej_points]
#     bone_num_list = [point_in_left_side_of_line(bone,line_of_inertia[0],line_of_inertia[1]) for bone in bone_points]
#     if sum(cej_num_list) == 1 and sum(bone_num_list) == 1:
#         L_side = True
#     if len(cej_points) - sum(cej_num_list) == 1 and len(bone_points) - sum(bone_num_list)==1:
#         R_side = True
#     #print("L_side:",L_side,"R_side:",R_side)
#     points_info = tuple((sum(cej_num_list),sum(bone_num_list),len(cej_points) - sum(cej_num_list),len(bone_points) - sum(bone_num_list)))
#     if print_:
#         print('L_side:',L_side,'R_side:',R_side)
#         print('roots_point:',roots_point)
#         print('bone_points:',bone_points)
#         print('cej_points:',cej_points)
#     if L_side and R_side:
#         cej_center_point = [int((cej_points[0][0]+cej_points[1][0])/2),int((cej_points[0][1]+cej_points[1][1])/2)]
#         cv2.line(img_PBL_line, (bone_points[0][0], bone_points[0][1]), (bone_points[1][0], bone_points[1][1]), (0, 255, 0), 5)
#         cv2.line(img_PBL_line, (cej_points[0][0], cej_points[0][1]), (cej_points[1][0], cej_points[1][1]), (255, 0, 0), 5)
#         cv2.line(img_PBL_line, (roots_point[0][0], roots_point[0][1]), (cej_center_point[0], cej_center_point[1]), (0, 0, 255), 5)
        
#         cv2.line(img_PBL_line, (cej_points[0][0],cej_points[0][1]), (bone_points[0][0], bone_points[0][1]), (0, 255, 0), 2)
#         cv2.line(img_PBL_line, (cej_points[1][0],cej_points[1][1]), (bone_points[1][0], bone_points[1][1]), (0, 255, 0), 2)
#         #cv2.line(img_PBL_line, (crosspoint_point2line(bone_points[0],cej_points[0],cej_points[1])), (bone_points[0][0], bone_points[0][1]), (0, 255, 0), 2)
#         #cv2.line(img_PBL_line, (crosspoint_point2line(bone_points[1],cej_points[0],cej_points[1])), (bone_points[1][0], bone_points[1][1]), (0, 255, 0), 2)
        
#         cv2.circle(img_PBL_line, (roots_point[0][0], roots_point[0][1]), 3 , (0, 0, 255), 10)
#         cv2.circle(img_PBL_line, (bone_points[0][0], bone_points[0][1]), 3 , (0, 255, 0), 10)
#         cv2.circle(img_PBL_line, (bone_points[1][0], bone_points[1][1]), 3 , (0, 255, 0), 10)
#         cv2.circle(img_PBL_line, (cej_points[0][0], cej_points[0][1]), 3 , (255, 0, 0), 10)
#         cv2.circle(img_PBL_line, (cej_points[1][0], cej_points[1][1]), 3 , (255, 0, 0), 10)
#         LPBL = (distance_point2line(np.array(bone_points[0]),np.array(cej_points[0]),np.array(cej_points[1]))*0.05-2)/(distance_2_points(np.array(roots_point[0]),np.array(cej_center_point))*0.05-2)
#         RPBL = (distance_point2line(np.array(bone_points[1]),np.array(cej_points[0]),np.array(cej_points[1]))*0.05-2)/(distance_2_points(np.array(roots_point[0]),np.array(cej_center_point))*0.05-2)

#         Dis_L = distance_point2line(np.array(bone_points[0]),np.array(cej_points[0]),np.array(cej_points[1]))
#         Dis_R = distance_point2line(np.array(bone_points[1]),np.array(cej_points[0]),np.array(cej_points[1]))
#         Dis_root = distance_2_points(np.array(roots_point[0]),np.array(cej_center_point))
#         if LPBL < 0:
#             LPBL = 0
#         if RPBL < 0:
#             RPBL = 0
#         return True, round(LPBL*100,3), PBL_stage(LPBL), round(RPBL*100,3), PBL_stage(RPBL),Dis_L,Dis_R,Dis_root, img_PBL_line,points_info
#     elif L_side:
#         RPBL = -999
#         cej_cross_point_with_inertia = crosspoint_point2line(cej_points[cej_num_list.index(True)],line_of_inertia[0],line_of_inertia[1])
#         bone_cross_point_with_inertia = crosspoint_point2line(bone_points[bone_num_list.index(True)],line_of_inertia[0],line_of_inertia[1])
#         cv2.line(img_PBL_line, (roots_point[0][0], roots_point[0][1]), (cej_cross_point_with_inertia[0], cej_cross_point_with_inertia[1]), (0, 0, 255), 5)
#         cv2.line(img_PBL_line, (cej_points[cej_num_list.index(True)][0], cej_points[cej_num_list.index(True)][1]), (cej_cross_point_with_inertia[0], cej_cross_point_with_inertia[1]), (255, 0, 0), 5)
#         cv2.line(img_PBL_line, (bone_points[bone_num_list.index(True)][0], bone_points[bone_num_list.index(True)][1]), (bone_cross_point_with_inertia[0], bone_cross_point_with_inertia[1]), (0, 255, 0), 5)
#         cv2.line(img_PBL_line, (cej_points[cej_num_list.index(True)][0],cej_points[cej_num_list.index(True)][1]), (bone_points[bone_num_list.index(True)][0], bone_points[bone_num_list.index(True)][1]), (0, 255, 0), 2)
#         LPBL = (distance_2_points(np.array(cej_points[cej_num_list.index(True)]),np.array(bone_points[bone_num_list.index(True)]))*0.05-2)/(distance_2_points(np.array(cej_cross_point_with_inertia),np.array(roots_point[0]))*0.05-2)
#         Dis_R = -999
#         Dis_L = distance_2_points(np.array(cej_points[cej_num_list.index(True)]),np.array(bone_points[bone_num_list.index(True)]))
#         Dis_root = distance_2_points(np.array(cej_cross_point_with_inertia),np.array(roots_point[0]))
#         if LPBL < 0:
#             LPBL = 0
#         return True, round(LPBL*100,3), PBL_stage(LPBL), RPBL, PBL_stage(RPBL),Dis_L,Dis_R,Dis_root,img_PBL_line,points_info
#     elif R_side:
#         LPBL = -999
#         cej_cross_point_with_inertia = crosspoint_point2line(cej_points[cej_num_list.index(False)],line_of_inertia[0],line_of_inertia[1])
#         bone_cross_point_with_inertia = crosspoint_point2line(bone_points[bone_num_list.index(False)],line_of_inertia[0],line_of_inertia[1])
#         cv2.line(img_PBL_line, (roots_point[0][0], roots_point[0][1]), (cej_cross_point_with_inertia[0], cej_cross_point_with_inertia[1]), (0, 0, 255), 5)
#         cv2.line(img_PBL_line, (cej_points[cej_num_list.index(False)][0], cej_points[cej_num_list.index(False)][1]), (cej_cross_point_with_inertia[0], cej_cross_point_with_inertia[1]), (255, 0, 0), 5)
#         cv2.line(img_PBL_line, (bone_points[bone_num_list.index(False)][0], bone_points[bone_num_list.index(False)][1]), (bone_cross_point_with_inertia[0], bone_cross_point_with_inertia[1]), (0, 255, 0), 5)
#         cv2.line(img_PBL_line, (cej_points[cej_num_list.index(False)][0],cej_points[cej_num_list.index(False)][1]), (bone_points[bone_num_list.index(False)][0], bone_points[bone_num_list.index(False)][1]), (0, 255, 0), 2)
#         RPBL = (distance_2_points(np.array(cej_points[cej_num_list.index(False)]),np.array(bone_points[bone_num_list.index(False)]))*0.05-2)/(distance_2_points(np.array(cej_cross_point_with_inertia),np.array(roots_point[0]))*0.05-2)
#         Dis_L = -999
#         Dis_R = distance_2_points(np.array(cej_points[cej_num_list.index(False)]),np.array(bone_points[bone_num_list.index(False)]))
#         Dis_root = distance_2_points(np.array(cej_cross_point_with_inertia),np.array(roots_point[0]))
#         if RPBL < 0:
#             RPBL = 0
#         return True, LPBL, PBL_stage(LPBL), round(RPBL*100,3), PBL_stage(RPBL),Dis_L,Dis_R,Dis_root,img_PBL_line,points_info
#     else:
#         LPBL, RPBL = -999, -999
#         Dis_L = -999
#         Dis_R = -999
#         Dis_root = -999
#         img_PBL_line_copy = img_PBL_line.copy()
#         fill_up_img = np.expand_dims(fill_up_img, axis=2)
#         fill_up_img = np.where(fill_up_img>0,1,0).astype('uint8')
#         cv2.line(img_PBL_line_copy, (line_of_inertia[0][0], line_of_inertia[0][1]), (line_of_inertia[1][0], line_of_inertia[1][1]), (0, 0, 255), 5)
#         img_PBL_line_copy*=fill_up_img
#         img_PBL_line*=abs(1-fill_up_img)
#         img_PBL_line += img_PBL_line_copy
#         return False,LPBL, PBL_stage(LPBL), RPBL, PBL_stage(RPBL),Dis_L,Dis_R,Dis_root,img_PBL_line,points_info

In [7]:
def PBL_Detect(fill_up_img, value, imgarray, roots_point, cej_points,bone_points, line_of_inertia, molar):
    print_ = False
    roots_point = list(sorted(roots_point, key = lambda x:x[0]))
    cej_points = list(sorted(cej_points, key = lambda x:x[0]))
    bone_points = list(sorted(bone_points, key = lambda x:x[0]))
    RGB_image = cv2.cvtColor(imgarray, cv2.COLOR_GRAY2RGB)
    img_PBL_line = RGB_image.copy()
    L_side = False
    R_side = False
    
    cej_num_list = [point_in_left_side_of_line(cej,line_of_inertia[0],line_of_inertia[1]) for cej in cej_points]
    bone_num_list = [point_in_left_side_of_line(bone,line_of_inertia[0],line_of_inertia[1]) for bone in bone_points]
    line_of_inertia_copy = line_of_inertia.copy()
    
    if sum(cej_num_list) == 1 and sum(bone_num_list) == 1:
        L_side = True
    if len(cej_points) - sum(cej_num_list) == 1 and len(bone_points) - sum(bone_num_list)==1:
        R_side = True
    #print("L_side:",L_side,"R_side:",R_side)
    
    LPBL, RPBL = -999, -999
    Dis_L = -999
    Dis_R = -999
    Dis_root = -999
    
    points_info = tuple((sum(cej_num_list),sum(bone_num_list),len(cej_points) - sum(cej_num_list),len(bone_points) - sum(bone_num_list)))
    if print_:
        print('L_side:',L_side,'R_side:',R_side)
        print('roots_point:',roots_point)
        print('bone_points:',bone_points)
        print('cej_points:',cej_points)
#     molar = False

#     if L_side and R_side and not molar:
#         cej_center_point = [int((cej_points[0][0]+cej_points[1][0])/2),int((cej_points[0][1]+cej_points[1][1])/2)]
#         cv2.line(img_PBL_line, (bone_points[0][0], bone_points[0][1]), (bone_points[1][0], bone_points[1][1]), (0, 255, 0), 5)
#         cv2.line(img_PBL_line, (cej_points[0][0], cej_points[0][1]), (cej_points[1][0], cej_points[1][1]), (255, 0, 0), 5)
#         cv2.line(img_PBL_line, (roots_point[0][0], roots_point[0][1]), (cej_center_point[0], cej_center_point[1]), (0, 0, 255), 5)

#         cv2.line(img_PBL_line, (cej_points[0][0],cej_points[0][1]), (bone_points[0][0], bone_points[0][1]), (0, 255, 0), 2)
#         cv2.line(img_PBL_line, (cej_points[1][0],cej_points[1][1]), (bone_points[1][0], bone_points[1][1]), (0, 255, 0), 2)
#         #cv2.line(img_PBL_line, (crosspoint_point2line(bone_points[0],cej_points[0],cej_points[1])), (bone_points[0][0], bone_points[0][1]), (0, 255, 0), 2)
#         #cv2.line(img_PBL_line, (crosspoint_point2line(bone_points[1],cej_points[0],cej_points[1])), (bone_points[1][0], bone_points[1][1]), (0, 255, 0), 2)

#         cv2.circle(img_PBL_line, (roots_point[0][0], roots_point[0][1]), 3 , (0, 0, 255), 10)
#         cv2.circle(img_PBL_line, (bone_points[0][0], bone_points[0][1]), 3 , (0, 255, 0), 10)
#         cv2.circle(img_PBL_line, (bone_points[1][0], bone_points[1][1]), 3 , (0, 255, 0), 10)
#         cv2.circle(img_PBL_line, (cej_points[0][0], cej_points[0][1]), 3 , (255, 0, 0), 10)
#         cv2.circle(img_PBL_line, (cej_points[1][0], cej_points[1][1]), 3 , (255, 0, 0), 10)
#         LPBL = (distance_point2line(np.array(bone_points[0]),np.array(cej_points[0]),np.array(cej_points[1]))*0.05-2)/(distance_2_points(np.array(roots_point[0]),np.array(cej_center_point))*0.05-2)
#         RPBL = (distance_point2line(np.array(bone_points[1]),np.array(cej_points[0]),np.array(cej_points[1]))*0.05-2)/(distance_2_points(np.array(roots_point[0]),np.array(cej_center_point))*0.05-2)

#         Dis_L = distance_point2line(np.array(bone_points[0]),np.array(cej_points[0]),np.array(cej_points[1]))
#         Dis_R = distance_point2line(np.array(bone_points[1]),np.array(cej_points[0]),np.array(cej_points[1]))
#         Dis_root = distance_2_points(np.array(roots_point[0]),np.array(cej_center_point))
#         if LPBL < 0:
#             LPBL = 0
#         if RPBL < 0:
#             RPBL = 0
#         return True, round(LPBL*100,3), PBL_stage(LPBL), round(RPBL*100,3), PBL_stage(RPBL),Dis_L,Dis_R,Dis_root, img_PBL_line,points_info
    if L_side:
        if molar:
            root_x, root_y, line_of_inertia = get_molar_root(fill_up_img, value, line_of_inertia, 'L')
            roots_point = list(sorted(zip(root_x, root_y), key = lambda x:x[0]))
            cej_num_list = [point_in_left_side_of_line(cej,line_of_inertia[0],line_of_inertia[1]) for cej in cej_points]
            bone_num_list = [point_in_left_side_of_line(bone,line_of_inertia[0],line_of_inertia[1]) for bone in bone_points]
            
        if sum(cej_num_list) == 1 and sum(bone_num_list) == 1:
            cej_cross_point_with_inertia = crosspoint_point2line(cej_points[cej_num_list.index(True)],line_of_inertia[0],line_of_inertia[1])
            bone_cross_point_with_inertia = crosspoint_point2line(bone_points[bone_num_list.index(True)],line_of_inertia[0],line_of_inertia[1])

            cv2.circle(img_PBL_line, (roots_point[0]), 5, (0, 0, 255), -1)
            cv2.line(img_PBL_line, (roots_point[0][0], roots_point[0][1]), (cej_cross_point_with_inertia[0], cej_cross_point_with_inertia[1]), (0, 0, 255), 5)
            cv2.line(img_PBL_line, (cej_points[cej_num_list.index(True)][0], cej_points[cej_num_list.index(True)][1]), (cej_cross_point_with_inertia[0], cej_cross_point_with_inertia[1]), (255, 0, 0), 5)
            cv2.line(img_PBL_line, (bone_points[bone_num_list.index(True)][0], bone_points[bone_num_list.index(True)][1]), (bone_cross_point_with_inertia[0], bone_cross_point_with_inertia[1]), (0, 255, 0), 5)
            cv2.line(img_PBL_line, (cej_points[cej_num_list.index(True)][0],cej_points[cej_num_list.index(True)][1]), (bone_points[bone_num_list.index(True)][0], bone_points[bone_num_list.index(True)][1]), (0, 255, 0), 2)

            LPBL = (distance_2_points(np.array(cej_points[cej_num_list.index(True)]),np.array(bone_points[bone_num_list.index(True)]))*0.05-2)/(distance_2_points(np.array(cej_cross_point_with_inertia),np.array(roots_point[0]))*0.05-2)
            Dis_L = distance_2_points(np.array(cej_points[cej_num_list.index(True)]),np.array(bone_points[bone_num_list.index(True)]))
            Dis_root = distance_2_points(np.array(cej_cross_point_with_inertia),np.array(roots_point[0]))
            if LPBL < 0:
                LPBL = 0
        else:
            pass

    if R_side:
        if molar:
            root_x, root_y, line_of_inertia = get_molar_root(fill_up_img, value, line_of_inertia_copy, 'R')
            roots_point = list(sorted(zip(root_x, root_y), key = lambda x:x[0]))
            cej_num_list = [point_in_left_side_of_line(cej,line_of_inertia[0],line_of_inertia[1]) for cej in cej_points]
            bone_num_list = [point_in_left_side_of_line(bone,line_of_inertia[0],line_of_inertia[1]) for bone in bone_points]
        if ((len(cej_num_list) - sum(cej_num_list)) == 1) and ((len(bone_points) - sum(bone_num_list))==1):      
            cej_cross_point_with_inertia = crosspoint_point2line(cej_points[cej_num_list.index(False)],line_of_inertia[0],line_of_inertia[1])
            bone_cross_point_with_inertia = crosspoint_point2line(bone_points[bone_num_list.index(False)],line_of_inertia[0],line_of_inertia[1])

            cv2.circle(img_PBL_line, (roots_point[0]), 5, (0, 0, 255), -1)
            cv2.line(img_PBL_line, (roots_point[0][0], roots_point[0][1]), (cej_cross_point_with_inertia[0], cej_cross_point_with_inertia[1]), (0, 0, 255), 5)
            cv2.line(img_PBL_line, (cej_points[cej_num_list.index(False)][0], cej_points[cej_num_list.index(False)][1]), (cej_cross_point_with_inertia[0], cej_cross_point_with_inertia[1]), (255, 0, 0), 5)
            cv2.line(img_PBL_line, (bone_points[bone_num_list.index(False)][0], bone_points[bone_num_list.index(False)][1]), (bone_cross_point_with_inertia[0], bone_cross_point_with_inertia[1]), (0, 255, 0), 5)
            cv2.line(img_PBL_line, (cej_points[cej_num_list.index(False)][0],cej_points[cej_num_list.index(False)][1]), (bone_points[bone_num_list.index(False)][0], bone_points[bone_num_list.index(False)][1]), (0, 255, 0), 2)

            RPBL = (distance_2_points(np.array(cej_points[cej_num_list.index(False)]),np.array(bone_points[bone_num_list.index(False)]))*0.05-2)/(distance_2_points(np.array(cej_cross_point_with_inertia),np.array(roots_point[0]))*0.05-2)
            Dis_R = distance_2_points(np.array(cej_points[cej_num_list.index(False)]),np.array(bone_points[bone_num_list.index(False)]))
            Dis_root = distance_2_points(np.array(cej_cross_point_with_inertia),np.array(roots_point[0]))
            if RPBL < 0:
                RPBL = 0
        else:
            pass
            
    if L_side or R_side:
        return True, max(round(LPBL*100,3), -999), PBL_stage(LPBL), max(round(RPBL*100,3), -999), PBL_stage(RPBL),Dis_L,Dis_R,Dis_root,img_PBL_line,points_info
    else:
        LPBL, RPBL = -999, -999
        Dis_L = -999
        Dis_R = -999
        Dis_root = -999
        img_PBL_line_copy = img_PBL_line.copy()
        fill_up_img = np.expand_dims(fill_up_img, axis=2)
        fill_up_img = np.where(fill_up_img>0,1,0).astype('uint8')
        cv2.line(img_PBL_line_copy, (line_of_inertia[0][0], line_of_inertia[0][1]), (line_of_inertia[1][0], line_of_inertia[1][1]), (0, 0, 255), 5)
        img_PBL_line_copy*=fill_up_img
        img_PBL_line*=abs(1-fill_up_img)
        img_PBL_line += img_PBL_line_copy
        return False,LPBL, PBL_stage(LPBL), RPBL, PBL_stage(RPBL),Dis_L,Dis_R,Dis_root,img_PBL_line,points_info

In [8]:
# #bone is follow to cej
# def keypoint_filter(cej_points, bone_points, line_of_inertia, value, fill_up_img):
#     new_list_x = []
#     new_list_y = []
#     cej_points = list(sorted(cej_points, key = lambda x:x[0]))
#     bone_points = list(sorted(bone_points, key = lambda x:x[0]))
    
# #     root_x, root_y, line_of_inertia_L = get_molar_root(fill_up_img, value, line_of_inertia, 'L')
# #     root_x, root_y, line_of_inertia_R = get_molar_root(fill_up_img, value, line_of_inertia, 'R')
    
# #     cej_num_list = [point_in_left_side_of_line(cej,line_of_inertia[0],line_of_inertia[1]) for cej in cej_points]
# #     bone_num_list = [point_in_left_side_of_line(bone,line_of_inertia[0],line_of_inertia[1]) for bone in bone_points]

#     root_x, root_y, line_of_inertia_L = get_molar_root(fill_up_img, value, line_of_inertia, 'L')
#     cej_num_list_L = [point_in_left_side_of_line(cej,line_of_inertia_L[0],line_of_inertia_L[1]) for cej in cej_points]
#     root_x, root_y, line_of_inertia_R = get_molar_root(fill_up_img, value, line_of_inertia, 'R')
#     cej_num_list_R = [point_in_left_side_of_line(cej,line_of_inertia_R[0],line_of_inertia_R[1]) for cej in cej_points]
    
# #     bone_num_list = [point_in_left_side_of_line(bone,line_of_inertia[0],line_of_inertia[1]) for bone in bone_points]
    
    
# #     cej_left_points = [cej for bool_left,cej in zip(cej_num_list,cej_points) if bool_left]
# #     cej_right_points = [cej for bool_left,cej in zip(cej_num_list,cej_points) if not bool_left]
#     cej_left_points = [cej for bool_left,cej in zip(cej_num_list_L,cej_points) if bool_left]
#     cej_right_points = [cej for bool_left,cej in zip(cej_num_list_R,cej_points) if not bool_left]
    
#     cej_x = []
#     cej_y = []
#     #upper tooth
#     if (value==1):
#         try:
#             cej_left_points = list(sorted(cej_left_points, key = lambda x:x[1]))[0]
#             cej_points.append(cej_left_points)
#             cej_x.append(cej_left_points[0])
#             cej_y.append(cej_left_points[1])
#         except:
#             pass
#         try:
#             cej_right_points = list(sorted(cej_right_points, key = lambda x:x[1]))[0]
#             cej_points.append(cej_right_points)
#             cej_x.append(cej_right_points[0])
#             cej_y.append(cej_right_points[1])
#         except:
#             pass
#     #lower tooth
#     if (value==0):
#         try:
#             cej_left_points = list(sorted(cej_left_points, key = lambda x:x[1],reverse=True))[0]
#             cej_points.append(cej_left_points)
#             cej_x.append(cej_left_points[0])
#             cej_y.append(cej_left_points[1])
#         except:
#             pass
#         try:
#             cej_right_points = list(sorted(cej_right_points, key = lambda x:x[1],reverse=True))[0]
#             cej_points.append(cej_right_points)
#             cej_x.append(cej_right_points[0])
#             cej_y.append(cej_right_points[1])
#         except:
#             pass
    
#     cej_final_points = [(x,y) for x,y in zip(cej_x,cej_y)]
#     cej_left_bool_list = [point_in_left_side_of_line(cej,line_of_inertia[0],line_of_inertia[1]) for cej in cej_final_points]
    
#     root_x, root_y, line_of_inertia_L = get_molar_root(fill_up_img, value, line_of_inertia, 'L')
#     bone_num_list_L = [point_in_left_side_of_line(bone, line_of_inertia_L[0], line_of_inertia_L[1]) for bone in bone_points]
#     root_x, root_y, line_of_inertia_R = get_molar_root(fill_up_img, value, line_of_inertia, 'R')
#     bone_num_list_R = [point_in_left_side_of_line(bone, line_of_inertia_R[0], line_of_inertia_R[1]) for bone in bone_points]
 
#     bone_left_points = [bone for bool_left,bone in zip(bone_num_list_L,bone_points) if  bool_left]
#     bone_right_points = [bone for bool_left,bone in zip(bone_num_list_R,bone_points) if not bool_left]
    
# #     bone_left_points = [bone for bool_left,bone in zip(bone_num_list,bone_points) if  bool_left]
# #     bone_right_points = [bone for bool_left,bone in zip(bone_num_list,bone_points) if not bool_left]
    
#     bone_x = []
#     bone_y = []
# #     print("-------------------------------")
# #     print("line_of_inertia[0],line_of_inertia[1]",line_of_inertia[0],line_of_inertia[1])
# #     print("cej_final_points:",cej_final_points)
# #     print("cej_final_side:",cej_num_list)
    
# #     print(len(bone_num_list))
# #     print("cej_num_list:",cej_num_list)
# #     print("cej_right_points:",cej_right_points)
# #     print("cej_left_points:",cej_left_points)
    
# #     print("bone_num_list:",bone_num_list)
# #     print("bone_right_points:",bone_right_points)
# #     print("bone_left_points:",bone_left_points)
# #     print("cej_x:",cej_x)
# #     print("cej_y:",cej_y)
    
#     #upper tooth
#     if (value==1):
#         try:
#             bone_left_points = list(sorted(bone_left_points, key = lambda x:x[1],reverse=True))
#             bone_left_points_exclude_on_crown = [(x,y) for (x,y) in bone_left_points if y < cej_y[cej_left_bool_list.index(True)]]
#             best_point = bone_left_points_exclude_on_crown[0]
#             bone_x.append(best_point[0])
#             bone_y.append(best_point[1])
#         except:
#             pass
#         try:
#             bone_right_points = list(sorted(bone_right_points, key = lambda x:x[1],reverse=True))
#             bone_right_points_exclude_on_crown = [(x,y) for (x,y) in bone_right_points if y < cej_y[cej_left_bool_list.index(False)]]
#             best_point = bone_right_points_exclude_on_crown[0]
#             bone_x.append(best_point[0])
#             bone_y.append(best_point[1])
#         except:
#             pass
#     #lower tooth
#     if (value==0):
#         try:
#             bone_left_points = list(sorted(bone_left_points, key = lambda x:x[1],reverse=False))
#             bone_left_points_exclude_on_crown = [(x,y) for (x,y) in bone_left_points if y > cej_y[cej_left_bool_list.index(True)]]
#             best_point = bone_left_points_exclude_on_crown[0]
#             bone_x.append(best_point[0])
#             bone_y.append(best_point[1])
#         except:
#             pass
#         try:
#             bone_right_points = list(sorted(bone_right_points, key = lambda x:x[1],reverse=False))
#             bone_right_points_exclude_on_crown = [(x,y) for (x,y) in bone_right_points if y > cej_y[cej_left_bool_list.index(False)]]
#             best_point = bone_right_points_exclude_on_crown[0]
#             bone_x.append(best_point[0])
#             bone_y.append(best_point[1])
#         except:
#             pass
# #     print("cej_x:",cej_x,"cej_y:",cej_y)
# #     print("bone_x:",bone_x,"bone_y:",bone_y)
# #     print("-------------------------------")
#     return (cej_x,cej_y),(bone_x,bone_y)

In [9]:
def PolygonArea(corners):
    n = len(corners) # of corners
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += corners[i][0] * corners[j][1]
        area -= corners[j][0] * corners[i][1]
    area = abs(area) / 2.0
    return area

def point_in_box(point_boxs_list,check_point):
    point_pair_list = [[point_boxs_list[i-1],point_boxs_list[i]] for i in range(len(point_boxs_list))]
    box_area = PolygonArea(point_boxs_list)
    count_area = 0
    for edge in point_pair_list:
        edge.append(check_point)
        count_area = PolygonArea(edge)+count_area
    if count_area - box_area < 1:
        return True
    else:
        return False
    
def get_molar_bool(json_file_path, check_point):
    with open(json_file_path, 'r') as file:
        json_file = json.load(file)
    for tooth in json_file['shapes']:
        check_in_box = point_in_box(tooth['points'],check_point)
        if check_in_box:
            tooth_num = int(tooth['label'].split('_')[0])
            if (tooth_num >= 1 and tooth_num <=3) or (tooth_num >= 14 and tooth_num <=19) or (tooth_num >= 30 and tooth_num <=32):
                return True
            else:
                return False
    return False

In [10]:
def get_molar_root(fill_up_img, value, line_of_inertia, type_):
    y_list,x_list = np.nonzero(fill_up_img)
    
    L_fill_up = fill_up_img.copy()
    if type_ == 'L':
        point_in_left = [point_in_left_side_of_line([x_,y_],line_of_inertia[0],line_of_inertia[1]) for (x_, y_) in zip(x_list,y_list)]
        for bool_, x_, y_ in zip(point_in_left, x_list, y_list):
            if (not bool_):
                L_fill_up[y_,x_] = 0
        root_x, root_y,line_of_inertia = root_point_detect(L_fill_up,value)
#         plt.imshow(L_fill_up)
#         plt.show()
        return root_x, root_y,line_of_inertia

    if type_ == 'R':
        R_fill_up = fill_up_img.copy()
        point_in_left = [point_in_left_side_of_line([x_,y_],line_of_inertia[0],line_of_inertia[1]) for (x_, y_) in zip(x_list,y_list)]
        for bool_, x_, y_ in zip(point_in_left, x_list, y_list):
            if bool_:
                R_fill_up[y_,x_] = 0          
        root_x, root_y,line_of_inertia = root_point_detect(R_fill_up,value)
#         plt.imshow(R_fill_up)
#         plt.show()
        return root_x, root_y,line_of_inertia

In [11]:
folder_build_path = './'

#Using filled up tooth & segmentation with cej levle to Pattern detection key point
o_image_folder = os.path.join(folder_build_path,'1_o_image')
o_image_path_list = glob.glob(o_image_folder+'/*.PNG')
o_image_path_list.sort()

seg_image_folder = os.path.join(folder_build_path,'1_seg_image')
seg_image_path_list = glob.glob(seg_image_folder+'/*.PNG')
seg_image_path_list.sort()

split_root_folder = os.path.join(folder_build_path,'1_roots_mask')
split_root_path_list = glob.glob(split_root_folder+'/*.PNG')
split_root_path_list.sort()

seg_img_with_cej_folder = os.path.join(folder_build_path,'2_seg_img_with_cej')
seg_img_with_cej_path_list = glob.glob(seg_img_with_cej_folder+'/*.PNG')
seg_img_with_cej_path_list.sort()

json_folder = os.path.join('/root/notebooks/tooth_thsis_experiment/PBL_detection/Data/json_file/*.json')
json_list = glob.glob(json_folder)
json_list.sort()

loss_root_count = 0

#save detection result,i command out the draw point on image, only save detection result
pattern_detection_result_save_path = './3_pattern_detection_result'
draw_all_points_on_o_img_save_path = './3_draw_all_points_on_o_img'
draw_all_points_on_seg_img_save_path = './3_draw_all_points_on_seg_img'

# if show,save = true would show,save image what this block create
PBL_status = False
show = False
save = True
CSV = True
save_file_type = '.PNG'
print_ = False
molar = True
count = 0
tooth_count = 0
seg_image_path_list = sorted(seg_image_path_list)
seg_image_path_list = seg_image_path_list[:end]


#iteration all image
for seg_image_path in tqdm(seg_image_path_list,total = len(seg_image_path_list)):
    #only name, it is a boring function like path.split('/')
    img_name = get_name_from_path(seg_image_path,False)
#Using in debug, find target name image
#     if 'NN_180821_160705_16ED77' not in img_name:
#         continue
        
    if True:
        if False:
            print(img_name) 
        #get root mask 
        roots_img_path_list = [root_name for root_name in split_root_path_list if img_name in root_name]
        
        #get o_img by replace folder
        o_img = cv2.imread(seg_image_path.replace('1_seg_image','1_o_image'),0)
        seg_img = cv2.imread(seg_image_path)
        
        #get tooth forward
        #get upper or lower tooth image,forward = True 
        #get upper value==1 & lower tooth value==0
        forward, value = tooth_forward(seg_img)

    #     if not forward:
#         plt.subplot(121)
#         plt.title('seg_img')
#         plt.imshow(seg_img)
#         plt.subplot(122)
#         plt.title('o_img')
#         plt.imshow(o_img, cmap = 'gray')
#         plt.show()
#         print(check_forward_status(value))

        if forward:
            #get json path
            target_json_file = [file_path for file_path in json_list if img_name in file_path]
            #count miss json number,and pass
            if len(target_json_file)==0:
                loss_root_count+=1
                continue
                
            #iteration allroot mask from (./1_roots_mask)
            for fill_up_path in roots_img_path_list:
                count+=1
                #get o_img to draw detection on that
                o_img_copy = o_img.copy()
                
                #get name
                fill_up_name = get_name_from_path(fill_up_path,True)
                
                #get image
                fill_up_img = cv2.imread(fill_up_path,0)
                
                #check shape same as o_img
                fill_up_img = cv2.resize(fill_up_img,o_img_copy.shape[:2][::-1],interpolation = cv2.INTER_NEAREST)
                
                #get segmentation with root mask
                seg_img_with_cej_img = cv2.imread(fill_up_path.replace('1_roots_mask','2_seg_img_with_cej'))
                
                #grt center point in root mask
                check_point = (np.mean(np.nonzero(fill_up_img)[1]),np.mean(np.nonzero(fill_up_img)[0]))
                
                #check the root is molar or not 
                molar = get_molar_bool(target_json_file[0], check_point)
                
                #!!!import!!!get cej_x,cej_y,bone_x,bone_y
                (cej_x,cej_y),(bone_x,bone_y) = find_cej_bone_keypoint(seg_img_with_cej_img)
                
                #get root point need value(tooth to know upper or lower)
                root_x, root_y,line_of_inertia = root_point_detect(fill_up_img,value)
                ####################################
                ####################################
                
                #!!!import!!!using line_of_inertia to filter the point
                (cej_x,cej_y),(bone_x,bone_y) = keypoint_filter(zip(cej_x,cej_y),zip(bone_x,bone_y),line_of_inertia,value,fill_up_img)
                
                
                #if miss cej point
                if (len(cej_x) == 1) and (len(cej_y) == 1):
                    (cej_x, cej_y) = cej_recover(cej_x, cej_y, line_of_inertia, fill_up_img)
                #if miss bone point
                if (len(bone_x) < 2) and (len(bone_y) < 2):
                    (bone_x, bone_y) = bone_recover(bone_x, bone_y, fill_up_img, line_of_inertia, seg_img_with_cej_img, value, molar)
                ####################################
                #################################### 
                
                #some root is empty, so cannot fill segmentation
                if root_x[0] != None:
                    if print_:
                        print('Find {} root point in this image'.format(len(root_x)))
                        print(root_x,root_y)
                        print('Find {} cej point in this image'.format(len(cej_x)))
                        print(cej_x,cej_y)
                        print('Find {} bone point in this image'.format(len(bone_x)))
                        print(bone_x,bone_y)
                    #input all info cej bone root point interia (to define the point is left side or right side) 
                    PBL_status, LPBL, L_Stage, RPBL, R_Stage,Dis_L,Dis_R,Dis_root,img_PBL_line,points_info = PBL_Detect(fill_up_img,value,o_img_copy,zip(root_x,root_y),zip(cej_x,cej_y),zip(bone_x,bone_y),line_of_inertia,molar)

                    if print_:
                        print('Dectation LPBL is : {}%'.format(LPBL))
                        print('Dectation RPBL is : {}%'.format(RPBL))
                    
                    #save image path 
                    tooth_pbl_line_path = os.path.join(pattern_detection_result_save_path,fill_up_name)
                    tooth_pbl_draw_all_points_on_o_img_path = os.path.join(draw_all_points_on_o_img_save_path,fill_up_name)
                    tooth_pbl_draw_all_points_on_seg_img_path = os.path.join(draw_all_points_on_seg_img_save_path,fill_up_name)
                    ####################
                    if CSV:
                        #save info in csv
                        # List of strings
                        #example:['Image_name', 'Tooth_num', 'LPBL', 'L_Stage', 'RPBL', 'R_Stage','tooth_pbl_line_path']
                        row_contents = [img_name, fill_up_name, LPBL, L_Stage , RPBL, R_Stage,Dis_L,Dis_R,Dis_root,points_info,tooth_pbl_line_path]

                        # Append a list as new line to an old csv file
                        append_list_as_row(csv_file_path, row_contents)
                    ####write points
                    #point have color so we turn grayscale image to RGB
                    o_img_copy = cv2.cvtColor(o_img_copy, cv2.COLOR_GRAY2RGB)
                    #draw point on image
                    for x,y in zip(root_x,root_y):
                        cv2.circle(o_img_copy,(x, y), 4, (0, 0, 255), -1)
                        cv2.circle(seg_img_with_cej_img,(x, y), 4, (255, 255, 255), -1)
                    for x,y in zip(bone_x,bone_y):
                        cv2.circle(o_img_copy,(x, y), 4, (0, 255, 0), -1)
                        cv2.circle(seg_img_with_cej_img,(x, y), 4, (255, 255, 0), -1)
                    for x,y in zip(cej_x,cej_y):
                        cv2.circle(o_img_copy,(x, y), 4, (255, 0, 0), -1)
                        cv2.circle(seg_img_with_cej_img,(x, y), 4, (0, 255, 255), -1)
                    ##############
                    if save:
                        #we only save image from function 'PBL_Detect'
                        cv2.imwrite(tooth_pbl_line_path, img_PBL_line)
#                         cv2.imwrite(tooth_pbl_draw_all_points_on_o_img_path, o_img_copy)
#                         cv2.imwrite(tooth_pbl_draw_all_points_on_seg_img_path, seg_img_with_cej_img)

                    #show result
                    if show and PBL_status:
                        plt.figure(figsize=(15,15))
                        plt.subplot(141)
                        plt.title('fill_up_img')
                        plt.imshow(fill_up_img, cmap = 'gray')
                        plt.subplot(142)
                        plt.title('seg_img_with_cej_img')
                        plt.imshow(cv2.cvtColor(seg_img_with_cej_img,cv2.COLOR_BGR2RGB))
                        plt.subplot(143)
                        plt.title('o_img')
                        plt.imshow(o_img_copy)
                        plt.subplot(144)
                        plt.title('img_PBL_line')
                        plt.imshow(cv2.cvtColor(img_PBL_line,cv2.COLOR_BGR2RGB))
                        plt.show()
                    elif show:
                        plt.figure(figsize=(12,12))
                        plt.subplot(131)
                        plt.title('fill_up_img')
                        plt.imshow(fill_up_img, cmap = 'gray')
                        plt.subplot(132)
                        plt.title('seg_img_with_cej_img')
                        plt.imshow(cv2.cvtColor(seg_img_with_cej_img,cv2.COLOR_BGR2RGB))
                        plt.subplot(133)
                        plt.title('o_img')
                        plt.imshow(cv2.cvtColor(o_img_copy,cv2.COLOR_BGR2RGB))
                        plt.show()
                else:
                    print("can't find the root", fill_up_name)
print('loss_root_count',loss_root_count)

can't find the root NN_070917_145730_16D49F_0.PNG


/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:356: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/root/notebooks/Prediction_final/PBL_detection/pattern_detection_tool.py:106: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov = np.cov(coords)
/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:2326: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:2326: RuntimeWarning: i

can't find the root NN_080825_101054_C0A926_0.PNG
can't find the root NN_080825_101154_C0A928_2.PNG
can't find the root NN_090218_155948_B4E32F_0.PNG
can't find the root NN_090218_160002_B4E330_1.PNG
can't find the root NN_090218_160002_B4E330_2.PNG
can't find the root NN_090218_160018_B4E33C_1.PNG
can't find the root NN_090218_160018_B4E33C_2.PNG
can't find the root NN_090218_160018_B4E33C_4.PNG
can't find the root NN_090218_160025_B4E331_0.PNG
can't find the root NN_090218_160025_B4E331_1.PNG
can't find the root NN_090218_160034_B4E33D_2.PNG
can't find the root NN_090218_160046_B4E333_1.PNG
can't find the root NN_090218_160046_B4E333_2.PNG
can't find the root NN_090218_160046_B4E333_3.PNG
can't find the root NN_090218_160053_B4E332_2.PNG
can't find the root NN_090218_162003_B4E33A_2.PNG
can't find the root NN_090218_162007_B4E336_0.PNG
can't find the root NN_090218_162007_B4E336_1.PNG
can't find the root NN_090218_162007_B4E336_3.PNG
can't find the root NN_090223_135828_C0372B_1.PNG


In [12]:
def point_in_left_side_of_line(point,line_edge1,line_edge2):
    dy = line_edge2[1]-line_edge1[1]
    dx = line_edge2[0]-line_edge1[0]
    if(dx==0):
        if(line_edge2[0]>point[0]):
            return True
        else:
            return False
    if(dy==0):
        dy = 1   
    try:
        m = (dy/dx)
    except:
        print('gradient error')
    c = line_edge1[1]-m*line_edge1[0]
    if (point[1] - (m*point[0]) - c) < 0:
        return True
    else:
        return False

In [13]:
line_of_inertia[0],line_of_inertia[1] (506, 1423) (369, -571)
cej_final_points: [(399, 480)]
cej_final_side: [False]
1
cej_num_list: [False]
cej_right_points: (399, 480)
cej_left_points: []
bone_num_list: [False]
bone_right_points: [(394, 452)]
bone_left_points: []
cej_x: [399]
cej_y: [480]
cej_x: [399] cej_y: [480]
bone_x: [394] bone_y: [452]
-------------------------------
L_side: False R_side: True

SyntaxError: invalid syntax (<ipython-input-13-ecec35ba2266>, line 13)

In [ ]:
point_in_left_side_of_line((0, 0),(506, 1423),(369, -571))

In [ ]:
point_in_left_side_of_line((0, 0),(4, 0),(0, 4))

In [ ]:
# def point_in_left_side_of_line(point,line_edge1,line_edge2):
#     try:
#         m = (line_edge2[1]-line_edge1[1])/(line_edge2[0]-line_edge1[0])
#     except:
#         m = (line_edge2[1]-line_edge1[1])/(1)
#     c = line_edge1[1]-m*line_edge1[0]
#     if (point[1] - c)/m >= point[0]:
#         return True
#     else:
#         return False
    

In [ ]:
line_of_inertia[0]: (253, 1318)
line_of_inertia[1]: (58, -672)
cej_points: [(75, 252)]
bone_points: [(219, 362)]

In [ ]:
img = np.ones((10,10,3),np.uint8)
purple = np.array([240, 32, 160])
red = np.array([0, 0, 255])
img = img*purple

img_gray = cv2.cvtColor(img.astype('uint8'),cv2.COLOR_BGR2GRAY)
print(img_gray[0][0])

In [ ]:
plt.imshow(cv2.cvtColor(img.astype('uint8'),cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
point_in_left_side_of_line((4,4),(0,4),(4,0))

In [ ]:
# def PBL_Detect(imgarray,roots_point,cej_points,bone_points,line_of_inertia):
#     print_ = False
#     roots_point = list(sorted(roots_point, key = lambda x:x[0]))
#     cej_points = list(sorted(cej_points, key = lambda x:x[0]))
#     bone_points = list(sorted(bone_points, key = lambda x:x[0]))
#     RGB_image = cv2.cvtColor(imgarray, cv2.COLOR_GRAY2RGB)
#     img_PBL_line = RGB_image.copy()
#     L_side = False
#     R_side = False
#     cej_num_list = [point_in_left_side_of_line(cej,line_of_inertia[0],line_of_inertia[1]) for cej in cej_points]
#     bone_num_list = [point_in_left_side_of_line(bone,line_of_inertia[0],line_of_inertia[1]) for bone in bone_points]
    
#     if sum(cej_num_list) == 1 and sum(bone_num_list) == 1:
#         R_side = True
#     if len(cej_points) - sum(cej_num_list) == 1 and len(bone_points) - sum(bone_num_list)==1:
#         L_side = True
#     points_info = tuple((sum(cej_num_list),sum(bone_num_list),len(cej_points) - sum(cej_num_list),len(bone_points) - sum(bone_num_list)))
#     if print_:
#         print('L_side:',L_side,'R_side:',R_side)
#         print('roots_point:',roots_point)
#         print('bone_points:',bone_points)
#         print('cej_points:',cej_points)
#     if L_side and R_side:
#         cej_center_point = [int((cej_points[0][0]+cej_points[1][0])/2),int((cej_points[0][1]+cej_points[1][1])/2)]
#         cv2.line(img_PBL_line, (bone_points[0][0], bone_points[0][1]), (bone_points[1][0], bone_points[1][1]), (0, 255, 0), 5)
#         cv2.line(img_PBL_line, (cej_points[0][0], cej_points[0][1]), (cej_points[1][0], cej_points[1][1]), (255, 0, 0), 5)
#         cv2.line(img_PBL_line, (roots_point[0][0], roots_point[0][1]), (cej_center_point[0], cej_center_point[1]), (0, 0, 255), 5)
        
#         cv2.line(img_PBL_line, (cej_points[0][0],cej_points[0][1]), (bone_points[0][0], bone_points[0][1]), (0, 255, 0), 2)
#         cv2.line(img_PBL_line, (cej_points[1][0],cej_points[1][1]), (bone_points[1][0], bone_points[1][1]), (0, 255, 0), 2)
#         #cv2.line(img_PBL_line, (crosspoint_point2line(bone_points[0],cej_points[0],cej_points[1])), (bone_points[0][0], bone_points[0][1]), (0, 255, 0), 2)
#         #cv2.line(img_PBL_line, (crosspoint_point2line(bone_points[1],cej_points[0],cej_points[1])), (bone_points[1][0], bone_points[1][1]), (0, 255, 0), 2)
        
#         cv2.circle(img_PBL_line, (roots_point[0][0], roots_point[0][1]), 3 , (0, 0, 255), 10)
#         cv2.circle(img_PBL_line, (bone_points[0][0], bone_points[0][1]), 3 , (0, 255, 0), 10)
#         cv2.circle(img_PBL_line, (bone_points[1][0], bone_points[1][1]), 3 , (0, 255, 0), 10)
#         cv2.circle(img_PBL_line, (cej_points[0][0], cej_points[0][1]), 3 , (255, 0, 0), 10)
#         cv2.circle(img_PBL_line, (cej_points[1][0], cej_points[1][1]), 3 , (255, 0, 0), 10)
#         LPBL = (distance_point2line(np.array(bone_points[0]),np.array(cej_points[0]),np.array(cej_points[1]))*0.05-2)/(distance_2_points(np.array(roots_point[0]),np.array(cej_center_point))*0.05-2)
#         RPBL = (distance_point2line(np.array(bone_points[1]),np.array(cej_points[0]),np.array(cej_points[1]))*0.05-2)/(distance_2_points(np.array(roots_point[0]),np.array(cej_center_point))*0.05-2)

#         Dis_L = distance_point2line(np.array(bone_points[0]),np.array(cej_points[0]),np.array(cej_points[1]))
#         Dis_R = distance_point2line(np.array(bone_points[1]),np.array(cej_points[0]),np.array(cej_points[1]))
#         Dis_root = distance_2_points(np.array(roots_point[0]),np.array(cej_center_point))
#         if LPBL < 0:
#             LPBL = 0
#         if RPBL < 0:
#             RPBL = 0
#         return True, round(LPBL*100,3), PBL_stage(LPBL), round(RPBL*100,3), PBL_stage(RPBL),Dis_L,Dis_R,Dis_root, img_PBL_line,points_info
#     elif L_side:
#         RPBL = -999
#         cej_cross_point_with_inertia = crosspoint_point2line(cej_points[cej_num_list.index(True)],line_of_inertia[0],line_of_inertia[1])
#         bone_cross_point_with_inertia = crosspoint_point2line(bone_points[bone_num_list.index(True)],line_of_inertia[0],line_of_inertia[1])
#         cv2.line(img_PBL_line, (roots_point[0][0], roots_point[0][1]), (cej_cross_point_with_inertia[0], cej_cross_point_with_inertia[1]), (0, 0, 255), 5)
#         cv2.line(img_PBL_line, (cej_points[cej_num_list.index(True)][0], cej_points[cej_num_list.index(True)][1]), (cej_cross_point_with_inertia[0], cej_cross_point_with_inertia[1]), (255, 0, 0), 5)
#         cv2.line(img_PBL_line, (bone_points[bone_num_list.index(True)][0], bone_points[bone_num_list.index(True)][1]), (bone_cross_point_with_inertia[0], bone_cross_point_with_inertia[1]), (0, 255, 0), 5)
#         cv2.line(img_PBL_line, (cej_points[cej_num_list.index(True)][0],cej_points[cej_num_list.index(True)][1]), (bone_points[bone_num_list.index(True)][0], bone_points[bone_num_list.index(True)][1]), (0, 255, 0), 2)
#         LPBL = (distance_2_points(np.array(cej_points[cej_num_list.index(True)]),np.array(bone_points[bone_num_list.index(True)]))*0.05-2)/(distance_2_points(np.array(cej_cross_point_with_inertia),np.array(roots_point[0]))*0.05-2)
#         Dis_R = -999
#         Dis_L = distance_2_points(np.array(cej_points[cej_num_list.index(True)]),np.array(bone_points[bone_num_list.index(True)]))
#         Dis_root = distance_2_points(np.array(cej_cross_point_with_inertia),np.array(roots_point[0]))
#         if LPBL < 0:
#             LPBL = 0
#         return True, round(LPBL*100,3), PBL_stage(LPBL), RPBL, PBL_stage(RPBL),Dis_L,Dis_R,Dis_root,img_PBL_line,points_info
#     elif R_side:
#         LPBL = -999
#         cej_cross_point_with_inertia = crosspoint_point2line(cej_points[cej_num_list.index(False)],line_of_inertia[0],line_of_inertia[1])
#         bone_cross_point_with_inertia = crosspoint_point2line(bone_points[bone_num_list.index(False)],line_of_inertia[0],line_of_inertia[1])
#         cv2.line(img_PBL_line, (roots_point[0][0], roots_point[0][1]), (cej_cross_point_with_inertia[0], cej_cross_point_with_inertia[1]), (0, 0, 255), 5)
#         cv2.line(img_PBL_line, (cej_points[cej_num_list.index(False)][0], cej_points[cej_num_list.index(False)][1]), (cej_cross_point_with_inertia[0], cej_cross_point_with_inertia[1]), (255, 0, 0), 5)
#         cv2.line(img_PBL_line, (bone_points[bone_num_list.index(False)][0], bone_points[bone_num_list.index(False)][1]), (bone_cross_point_with_inertia[0], bone_cross_point_with_inertia[1]), (0, 255, 0), 5)
#         cv2.line(img_PBL_line, (cej_points[cej_num_list.index(False)][0],cej_points[cej_num_list.index(False)][1]), (bone_points[bone_num_list.index(False)][0], bone_points[bone_num_list.index(False)][1]), (0, 255, 0), 2)
#         RPBL = (distance_2_points(np.array(cej_points[cej_num_list.index(False)]),np.array(bone_points[bone_num_list.index(False)]))*0.05-2)/(distance_2_points(np.array(cej_cross_point_with_inertia),np.array(roots_point[0]))*0.05-2)
#         Dis_L = -999
#         Dis_R = distance_2_points(np.array(cej_points[cej_num_list.index(False)]),np.array(bone_points[bone_num_list.index(False)]))
#         Dis_root = distance_2_points(np.array(cej_cross_point_with_inertia),np.array(roots_point[0]))
#         if RPBL < 0:
#             RPBL = 0
#         return True, LPBL, PBL_stage(LPBL), round(RPBL*100,3), PBL_stage(RPBL),Dis_L,Dis_R,Dis_root,img_PBL_line,points_info
#     else:
#         LPBL, RPBL = -999, -999
#         Dis_L = -999
#         Dis_R = -999
#         Dis_root = -999
#         cv2.line(img_PBL_line, (line_of_inertia[0][0], line_of_inertia[0][1]), (line_of_inertia[1][0], line_of_inertia[1][1]), (0, 0, 255), 5)
#         return False,LPBL, PBL_stage(LPBL), RPBL, PBL_stage(RPBL),Dis_L,Dis_R,Dis_root,img_PBL_line,points_info

In [ ]:
#bone level is cloest to root
# def keypoint_filter(cej_points,bone_points,line_of_inertia,value):
#     new_list_x = []
#     new_list_y = []
#     cej_points = list(sorted(cej_points, key = lambda x:x[0]))
#     bone_points = list(sorted(bone_points, key = lambda x:x[0]))
#     cej_num_list = [point_in_left_side_of_line(cej,line_of_inertia[0],line_of_inertia[1]) for cej in cej_points]
#     bone_num_list = [point_in_left_side_of_line(bone,line_of_inertia[0],line_of_inertia[1]) for bone in bone_points]
#     cej_right_points = [cej for bool_left,cej in zip(cej_num_list,cej_points) if not bool_left]
#     cej_left_points = [cej for bool_left,cej in zip(cej_num_list,cej_points) if bool_left]
#     cej_x = []
#     cej_y = []
#     #upper tooth
#     if (value==1):
#         try:
#             cej_left_points = list(sorted(cej_left_points, key = lambda x:x[1]))[0]
#             cej_points.append(cej_left_points)
#             cej_x.append(cej_left_points[0])
#             cej_y.append(cej_left_points[1])
#         except:
#             pass
#         try:
#             cej_right_points = list(sorted(cej_right_points, key = lambda x:x[1]))[0]
#             cej_points.append(cej_right_points)
#             cej_x.append(cej_right_points[0])
#             cej_y.append(cej_right_points[1])
#         except:
#             pass
#     #lower tooth
#     if (value==0):
#         try:
#             cej_left_points = list(sorted(cej_left_points, key = lambda x:x[1],reverse=True))[0]
#             cej_points.append(cej_left_points)
#             cej_x.append(cej_left_points[0])
#             cej_y.append(cej_left_points[1])
#         except:
#             pass
#         try:
#             cej_right_points = list(sorted(cej_right_points, key = lambda x:x[1],reverse=True))[0]
#             cej_points.append(cej_right_points)
#             cej_x.append(cej_right_points[0])
#             cej_y.append(cej_right_points[1])
#         except:
#             pass

#     bone_right_points = [bone for bool_right,bone in zip(bone_num_list,bone_points) if bool_right]
#     bone_left_points = [bone for bool_right,bone in zip(bone_num_list,bone_points) if not bool_right]
#     bone_x = []
#     bone_y = []
#     #upper tooth
#     if (value==1):
#         try:
#             bone_left_points = list(sorted(bone_left_points, key = lambda x:x[1]))[0]
#             bone_x.append(bone_left_points[0])
#             bone_y.append(bone_left_points[1])
#         except:
#             pass
#         try:
#             bone_right_points = list(sorted(bone_right_points, key = lambda x:x[1]))[0]
#             bone_x.append(bone_right_points[0])
#             bone_y.append(bone_right_points[1])
#         except:
#             pass
#     #lower tooth
#     if (value==0):
#         try:
#             bone_left_points = list(sorted(bone_left_points, key = lambda x:x[1],reverse=True))[0]
#             bone_x.append(bone_left_points[0])
#             bone_y.append(bone_left_points[1])
#         except:
#             pass
#         try:
#             bone_right_points = list(sorted(bone_right_points, key = lambda x:x[1],reverse=True))[0]
#             bone_x.append(bone_right_points[0])
#             bone_y.append(bone_right_points[1])
#         except:
#             pass
#     return (cej_x,cej_y),(bone_x,bone_y)

In [ ]:
# bone level is cloest to cej
# def keypoint_filter(cej_points,bone_points,line_of_inertia,value):
#     new_list_x = []
#     new_list_y = []
#     cej_points = list(sorted(cej_points, key = lambda x:x[0]))
#     bone_points = list(sorted(bone_points, key = lambda x:x[0]))
#     cej_num_list = [point_in_left_side_of_line(cej,line_of_inertia[0],line_of_inertia[1]) for cej in cej_points]
#     bone_num_list = [point_in_left_side_of_line(bone,line_of_inertia[0],line_of_inertia[1]) for bone in bone_points]
    
#     cej_left_points = [cej for bool_left,cej in zip(cej_num_list,cej_points) if bool_left]
#     cej_right_points = [cej for bool_left,cej in zip(cej_num_list,cej_points) if not bool_left]
    
#     cej_x = []
#     cej_y = []
#     #upper tooth
#     if (value==1):
#         try:
#             cej_left_points = list(sorted(cej_left_points, key = lambda x:x[1]))[0]
#             cej_points.append(cej_left_points)
#             cej_x.append(cej_left_points[0])
#             cej_y.append(cej_left_points[1])
#         except:
#             pass
#         try:
#             cej_right_points = list(sorted(cej_right_points, key = lambda x:x[1]))[0]
#             cej_points.append(cej_right_points)
#             cej_x.append(cej_right_points[0])
#             cej_y.append(cej_right_points[1])
#         except:
#             pass
#     #lower tooth
#     if (value==0):
#         try:
#             cej_left_points = list(sorted(cej_left_points, key = lambda x:x[1],reverse=True))[0]
#             cej_points.append(cej_left_points)
#             cej_x.append(cej_left_points[0])
#             cej_y.append(cej_left_points[1])
#         except:
#             pass
#         try:
#             cej_right_points = list(sorted(cej_right_points, key = lambda x:x[1],reverse=True))[0]
#             cej_points.append(cej_right_points)
#             cej_x.append(cej_right_points[0])
#             cej_y.append(cej_right_points[1])
#         except:
#             pass
    
#     cej_final_points = [(x,y) for x,y in zip(cej_x,cej_y)]
#     cej_left_bool_list = [point_in_left_side_of_line(cej,line_of_inertia[0],line_of_inertia[1]) for cej in cej_final_points]
    
#     bone_left_points = [bone for bool_left,bone in zip(bone_num_list,bone_points) if  bool_left]
#     bone_right_points = [bone for bool_left,bone in zip(bone_num_list,bone_points) if not bool_left]
    
#     bone_x = []
#     bone_y = []
# #     print("-------------------------------")
# #     print("line_of_inertia[0],line_of_inertia[1]",line_of_inertia[0],line_of_inertia[1])
# #     print("cej_final_points:",cej_final_points)
# #     print("cej_final_side:",cej_num_list)
    
# #     print(len(bone_num_list))
# #     print("cej_num_list:",cej_num_list)
# #     print("cej_right_points:",cej_right_points)
# #     print("cej_left_points:",cej_left_points)
    
# #     print("bone_num_list:",bone_num_list)
# #     print("bone_right_points:",bone_right_points)
# #     print("bone_left_points:",bone_left_points)
# #     print("cej_x:",cej_x)
# #     print("cej_y:",cej_y)
    
#     #upper tooth
#     if (value==1):
#         try:
#             bone_left_points = list(sorted(bone_left_points, key = lambda x:x[1],reverse=True))
#             point_distance_from_cej = [distance_2_points(point,[cej_x[cej_left_bool_list.index(True)],cej_y[cej_left_bool_list.index(True)]]) for point in bone_left_points]
#             best_point = bone_left_points[np.argmin(np.array(point_distance_from_cej))]
#             bone_x.append(best_point[0])
#             bone_y.append(best_point[1])
#         except:
#             pass
#         try:
#             bone_right_points = list(sorted(bone_right_points, key = lambda x:x[1],reverse=False))
#             point_distance_from_cej = [distance_2_points(point,[cej_x[cej_left_bool_list.index(False)],cej_y[cej_left_bool_list.index(False)]]) for point in bone_right_points]
#             best_point = bone_right_points[np.argmin(np.array(point_distance_from_cej))]
#             bone_x.append(best_point[0])
#             bone_y.append(best_point[1])
#         except:
#             pass
#     #lower tooth
#     if (value==0):
#         try:
#             bone_left_points = list(sorted(bone_left_points, key = lambda x:x[1],reverse=True))
#             point_distance_from_cej = [distance_2_points(point,[cej_x[cej_left_bool_list.index(True)],cej_y[cej_left_bool_list.index(True)]]) for point in bone_left_points]
#             best_point = bone_left_points[np.argmin(np.array(point_distance_from_cej))]
#             bone_x.append(best_point[0])
#             bone_y.append(best_point[1])
#         except:
#             pass
#         try:
#             bone_right_points = list(sorted(bone_right_points, key = lambda x:x[1],reverse=False))
#             point_distance_from_cej = [distance_2_points(point,[cej_x[cej_left_bool_list.index(False)],cej_y[cej_left_bool_list.index(False)]]) for point in bone_right_points]
#             best_point = bone_right_points[np.argmin(np.array(point_distance_from_cej))]
#             bone_x.append(best_point[0])
#             bone_y.append(best_point[1])
#         except:
#             pass
# #     print("cej_x:",cej_x,"cej_y:",cej_y)
# #     print("bone_x:",bone_x,"bone_y:",bone_y)
# #     print("-------------------------------")
#     return (cej_x,cej_y),(bone_x,bone_y)